In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from functools import reduce
import re
import matplotlib.pyplot as plt
import datetime

In [3]:
df_main=pd.read_csv('data.csv')
#df_main=df_main.drop['Unamed']
df_main=df_main.drop('Unnamed: 0', axis=1)
df_main

,Date,ST,CB,PB,GO,CA
0,2020-01-01,194.59,79.43,259.95,161.41,96.39
1,2020-01-02,194.59,77.58,253.28,143.95,96.85
2,2020-01-03,194.12,78.03,255.12,145.86,96.84
3,2020-01-06,193.81,77.50,254.16,147.39,96.67
4,2020-01-07,194.30,77.73,254.36,147.97,97.00
...,...,...,...,...,...,...
279,2020-12-25,194.59,79.43,259.95,161.41,90.32
280,2020-12-28,204.02,80.86,253.55,175.71,90.34
281,2020-12-29,204.00,80.86,253.82,176.35,89.99
282,2020-12-30,203.32,80.42,253.84,177.70,89.68


In [31]:
def month_price(n_months,date_in):
    delta = datetime.timedelta(days=1)
    frames=[]
    for i in range(n_months):
        filtered_df = df_main.loc[df_main['Date'] == str(date_in)]
        while (filtered_df.index.size < 1):
            filtered_df = df_main.loc[df_main['Date'] == str(date_in)]
            date_in+=delta

        frames.append(filtered_df)
    return frames

In [60]:
money_in=100
date_in="2020-01-01"
date_out="2020-06-15"
#amount should be equal to money_in*allocation
allocation=np.array([.5, .2, .2, .1, 0])
value_t0=money_in*allocation
amount=np.array([50, 20, 0, 30, 0])

#Date in datetime format
m=re.compile(r'\d+')
date_in=m.findall(date_in)
date_in=np.array(date_in).astype(int)

m=re.compile(r'\d+')
date_out=m.findall(date_out)
date_out=np.array(date_out).astype(int)

date_in= datetime.date(date_in[0],date_in[1],date_in[2])
date_out=datetime.date(date_out[0],date_out[1],date_out[2])

### BUY

In [61]:
n_months = ((date_out.year - date_in.year) * 12 + (date_out.month - date_in.month))
if(date_in.day>date_out.day):
    n_months-=1
    
#first day next month
date_in = (date_in.replace(day=1) + datetime.timedelta(days=32)).replace(day=1)

#cycle and store the price of the different periods in a df
delta = datetime.timedelta(days=1)
frames=[]
for i in range(n_months):
    filtered_df = df_main.loc[df_main['Date'] == str(date_in)]
    while (filtered_df.index.size < 1):
        filtered_df = df_main.loc[df_main['Date'] == str(date_in)]
        date_in+=delta
    
    frames.append(filtered_df)
    date_in = (date_in.replace(day=1) + datetime.timedelta(days=32)).replace(day=1)


months_price=pd.concat(frames)
periods=months_price['Date']

months_price=months_price.iloc[:,1:]#don't forget to change price of cash =1


#now work with weigh
amount_per_month=amount/n_months #(10,4,6,0)

shares = amount_per_month/months_price
print("shares: ",shares)
value_t0 =shares*months_price

print(value_t0)


shares:             ST        CB   PB        GO   CA
23   0.051390  0.050359  0.0  0.040442  0.0
43   0.054819  0.049164  0.0  0.040214  0.0
65   0.066684  0.052070  0.0  0.040147  0.0
87   0.051390  0.050391  0.0  0.037552  0.0
112  0.051390  0.049249  0.0  0.036661  0.0
       ST   CB   PB   GO   CA
23   10.0  4.0  0.0  6.0  0.0
43   10.0  4.0  0.0  6.0  0.0
65   10.0  4.0  0.0  6.0  0.0
87   10.0  4.0  0.0  6.0  0.0
112  10.0  4.0  0.0  6.0  0.0


### REBALANCING

In [53]:
def rebalancing(date_in,shares,allocation,value_t0,n_months):

    date_in = (date_in.replace(day=15) )
    print('date considered is now: ',date_in)    

    price=month_price(1,date_in)  
    price=price[0]
    price_out=price.iloc[:,1:]
    #print(price_out)

    percentage_t0=allocation#np.array([50, 20, 0, 30, 0])#  

    value_t1=np.multiply(shares, price_out)


    sum_value_t1=float(value_t1.sum(axis=1))#103,1

    percentage_t1=((value_t1*(100/n_months))/sum_value_t1)/100
    #print("percentage_t0: ",percentage_t0)
    #print("percentage_t1: ", percentage_t1)

    value_tr2=value_t1*percentage_t0/percentage_t1

    #print("value tr2: ",value_tr2)
    sum_value_tr2=float(value_tr2.sum(axis=1))#103,1

    percentage_tr2=((value_tr2*(100/n_months))/sum_value_tr2)/100
    #print("percentage tr2: ",percentage_tr2)

    data={'Asset':['ST', 'CB', 'PB', 'GO', 'CA'],
         '$_t0':value_t0,
         '%_t0':percentage_t0,
         '$_t1':value_t1.iloc[0],
         '%_t1':percentage_t1.iloc[0],
         '$_tr2':value_tr2.iloc[0],
         '%_tr2':percentage_tr2.iloc[0],
          }

    portfolio_rebalancing=pd.DataFrame(data,columns=['$_t0','%_t0','$_t1','%_t1','$_tr2','%_tr2']).fillna(0)
    print( portfolio_rebalancing)
#print(date_in)
#it works actually but we got some problem with cash 



In [55]:
i=0
value_t0=(money_in*amount_per_month)/100
for month in periods:
    m=re.compile(r'\d+')
    date_in=m.findall(month)
    date_in=np.array(date_in).astype(int)
    date_in= datetime.date(date_in[0],date_in[1],date_in[2])
    print('date of the investment', date_in)
    rebalancing(date_in,shares.iloc[i],(amount_per_month/100),value_t0,5)
    i+=1

date of the investment 2020-02-03
date considered is now:  2020-02-15
    $_t0  %_t0       $_t1      %_t1      $_tr2  %_tr2
ST  10.0  0.10  10.581736  0.102399  10.333871   0.10
CB   4.0  0.04   3.982878  0.038542   4.133548   0.04
PB   0.0  0.00   0.000000  0.000000   0.000000   0.00
GO   6.0  0.06   6.103128  0.059059   6.200322   0.06
CA   0.0  0.00   0.000000  0.000000   0.000000   0.00
date of the investment 2020-03-02
date considered is now:  2020-03-15
    $_t0  %_t0      $_t1      %_t1    $_tr2  %_tr2
ST  10.0  0.10  7.693784  0.089804  8.56730   0.10
CB   4.0  0.04  3.744838  0.043711  3.42692   0.04
PB   0.0  0.00  0.000000  0.000000  0.00000   0.00
GO   6.0  0.06  5.695979  0.066485  5.14038   0.06
CA   0.0  0.00  0.000000  0.000000  0.00000   0.00
date of the investment 2020-04-01
date considered is now:  2020-04-15
    $_t0  %_t0       $_t1      %_t1      $_tr2  %_tr2
ST  10.0  0.10  11.038944  0.101914  10.831649   0.10
CB   4.0  0.04   4.126530  0.038097   4.332660   0.0